# Tir balistique avec frottements
Le but est de déterminer la portée d'un tir dans un champ de pesanteur ($g=9.81 m/s^2$) en tenant compte des frottements. On testera deux types de frottements (cas d'un ballon de football) :
* des frottements fluides linéaires (coefficient $l = 3.5 \times 10^{-5} kg.s^{-1}$)
* des frottements fluides quadratiques (coefficient $k = 8.3 \times 10^{-3} kg.m^{-1}$)

Le mobile est tiré depuis le point $O(0, 0)$ avec une vitesse $v_0$ selon un angle $\alpha$.

La mise en équation donne donc :

* cas linéaire :

$${\rm{d} \over \rm{d}t}
\begin{pmatrix}
x\\
z\\
\dot x\\
\dot z
\end{pmatrix}
=
\begin{pmatrix}
\dot x\\
\dot z\\
- \frac{l}{m} \dot x\\
- \frac{l}{m} \dot z - g
\end{pmatrix}
$$

* cas quadratique :

$${\rm{d} \over \rm{d}t}
\begin{pmatrix}
x\\
z\\
\dot x\\
\dot z
\end{pmatrix}
=
\begin{pmatrix}
\dot x\\
\dot z\\
- \frac{l}{m} \dot x \sqrt{\dot x^2 + \dot z^2}\\
- \frac{l}{m} \dot z \sqrt{\dot x^2 + \dot z^2}\\ - g
\end{pmatrix}
$$

avec 

$$
\begin{pmatrix}
x\\
z\\
\dot x\\
\dot z
\end{pmatrix}
(t=0) =
\begin{pmatrix}
0\\
0\\
v_0 \cos \alpha\\
v_0 \sin \alpha
\end{pmatrix}
$$

On implémentera la vision vectorielle du schéma d'Euler explicite :

$$
Y_{k+1} = Y_k + F(Y_k, t_k) \times pas
$$

avec :

$$
Y_k =
\begin{pmatrix}
x_k\\
z_k\\
\dot x_k\\
\dot z_k
\end{pmatrix}
$$

> __Exercice :__  
> 1. Ecrire les fonctions `Flin` et `Fquad` associée au schéma d'Euler explicite puis la fonction `euler(F, Y0, tf, pas)` qui réalise l'intégration de $t=0$ à $t=tf$ avec un `pas` d'intégration.
> 2. Utiliser la fonction précédente pour obtenir la trajectoire pour un angle $\alpha = \pi / 4$ et une vitesse initiale $v_0 = 10 m/s$. Les représenter et lire graphiquement la portée dans les deux cas.
> 3. Ecrire une fonction `portee(F, Y0)` qui déterminer la portée du tir pour une fonction `F` (distance suivant x parcouru quand le mobile revient àl l'altitude $z=0$). Tracer alors pour des frottements linéaires, puis pour des frottements quadratiques la portée en fonction de l'angle pour une vitesse $v_0$ donnée. Quel angle (par elcture graphique) permet une portée maximale ? Retrouver cette valeur en utilisant les fonctions de Python.